In [1]:
# 基本パッケージ（numpy,Pandas,matplotlib）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 線形サポートベクターマシーン
from sklearn.svm import LinearSVC
# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
# train_test_split（データを分割出してくれる）
from sklearn.model_selection import train_test_split
# accuracy_score（正解率を測れる）
from sklearn.metrics import accuracy_score
# グリッドサーチ（ハイパーパラメータを自動的に最適化してくれる）
from sklearn.model_selection import GridSearchCV
# 正規化
from sklearn.preprocessing import MinMaxScaler
# 交差検証用
from sklearn.model_selection import cross_val_score
# 交互作用特徴量、多項式特徴量生成
from sklearn.preprocessing import PolynomialFeatures
# モデルの保存
from sklearn.externals import joblib

D:\Program\Anaconda3\envs\sklearn1.9.1\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)

In [3]:
# 株価データの読み込み
stock_data = pd.read_csv("stock_Price_Prediction_v2.4.csv", encoding="shift-jis")

# 読み込みデータの表示
stock_data.head()

,Low,Adj Close,CAD,CHF,AUD,IDR(100),INR,PHP,KRW(100),THB,KWD,TWD,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,answer,"SEK_bin_(-0.068, -0.0105]","SEK_bin_(-0.0105, -0.00609]","SEK_bin_(-0.00609, -0.00346]","SEK_bin_(-0.00346, -0.00154]","SEK_bin_(-0.00154, 0.000598]","SEK_bin_(0.000598, 0.00221]","SEK_bin_(0.00221, 0.00413]","SEK_bin_(0.00413, 0.00655]","SEK_bin_(0.00655, 0.01]","SEK_bin_(0.01, 0.0825]","SGD_bin_(-0.0473, -0.00699]","SGD_bin_(-0.00699, -0.00403]","SGD_bin_(-0.00403, -0.00225]","SGD_bin_(-0.00225, -0.000919]","SGD_bin_(-0.000919, 0.000282]","SGD_bin_(0.000282, 0.00138]","SGD_bin_(0.00138, 0.00265]","SGD_bin_(0.00265, 0.00424]","SGD_bin_(0.00424, 0.00671]","SGD_bin_(0.00671, 0.0634]","Volume_bin_(-0.927, -0.512]","Volume_bin_(-0.512, -0.388]","Volume_bin_(-0.388, -0.276]","Volume_bin_(-0.276, -0.155]","Volume_bin_(-0.155, -0.0187]","Volume_bin_(-0.0187, 0.133]","Volume_bin_(0.133, 0.32]","Volume_bin_(0.32, 0.626]","Volume_bin_(0.626, 1.115]","Volume_bin_(1.115, 11.805]","Open_bin_(-0.09620000000000001, -0.0162]","Open_bin_(-0.0162, -0.00927]","Open_bin_(-0.00927, -0.00528]","Open_bin_(-0.00528, -0.00222]","Open_bin_(-0.00222, 0.000726]","Open_bin_(0.000726, 0.00342]","Open_bin_(0.00342, 0.00639]","Open_bin_(0.00639, 0.0102]","Open_bin_(0.0102, 0.0159]","Open_bin_(0.0159, 0.16]","ZAR_bin_(-0.155, -0.0148]","ZAR_bin_(-0.0148, -0.00856]","ZAR_bin_(-0.00856, -0.00495]","ZAR_bin_(-0.00495, -0.00209]","ZAR_bin_(-0.00209, 0.000597]","ZAR_bin_(0.000597, 0.00299]","ZAR_bin_(0.00299, 0.00571]","ZAR_bin_(0.00571, 0.00916]","ZAR_bin_(0.00916, 0.0142]","ZAR_bin_(0.0142, 0.129]",SEK,SGD,Volume,Open,ZAR,High,GBP,SEK^2,SEK SGD,SEK Volume,SEK Open,SEK ZAR,SEK High,SEK GBP,SGD^2,SGD Volume,SGD Open,SGD ZAR,SGD High,SGD GBP,Volume^2,Volume Open,Volume ZAR,Volume High,Volume GBP,Open^2,Open ZAR,Open High,Open GBP,ZAR^2,ZAR High,ZAR GBP,High^2,High GBP,GBP^2
0,0.004525,0.010801,-0.001198,0.010974,-0.001687,0.007299,0.000000,0.003831,0.000000,-0.003257,0.002004,0.000000,0,1,0,0,0,1.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.010101,-0.000415,-0.514297,-0.004444,0.001701,0.000000,0.010485,0.000102,-0.000004,-0.005195,-0.000045,0.000017,0.000000,0.000106,1.726028e-07,0.000214,0.000002,-7.065560e-07,-0.000000,-0.000004,0.264501,0.002286,-0.000875,-0.000000,-0.005393,0.000020,-0.000008,-0.000000,-0.000047,0.000003,0.000000,0.000018,0.000000,0.000000,1.099410e-04
1,-0.003604,0.018700,0.003238,-0.001622,0.001690,-0.007246,0.003663,0.000000,-0.008893,-0.006536,0.003609,0.000000,0,0,1,0,0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-0.003846,-0.000831,2.643314,-0.006250,0.009338,0.024889,-0.002711,0.000015,0.000003,-0.010167,0.000024,-0.000036,-0.000096,0.000010,6.909852e-07,-0.002197,0.000005,-7.762145e-06,-0.000021,0.000002,6.987108,-0.016521,0.024683,0.065789,-0.007167,0.000039,-0.000058,-0.000156,0.000017,0.000087,0.000232,-0.000025,0.000619,-0.000067,7.351908e-06
2,0.028933,0.000874,0.000837,0.002000,-0.004639,0.000000,-0.003650,-0.003817,-0.000997,0.003289,-0.003367,0.000000,0,0,0,1,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-0.001544,0.001941,0.208818,0.026954,0.010934,0.004337,-0.000680,0.000002,-0.000003,-0.000322,-0.000042,-0.000017,-0.000007,0.000001,3.768293e-06,0.000405,0.000052,2.122432e-05,0.000008,-0.000001,0.043605,0.005629,0.002283,0.000906,-0.000142,0.000727,0.000295,0.000117,-0.000018,0.000120,0.000047,-0.000007,0.000019,-0.000003,4.619962e-07
3,-0.002636,-0.003493,-0.009078,-0.008855,-0.006779,0.000000,-0.007326,-0.003831,0.000998,-0.003279,-0.006665,-0.005236,0,0,0,0,1,1.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-0.004640,-0.005674,-0.317789,0.000000,-0.015807,-0.011226,-0.007273,0.000022,0.000026,0.001475,-0.000000,0.000073,0.000052,0.000034,3.219377e-05,0.001803,-0.000000,8.

In [4]:
# DataFrameをNumPy配列へ変換
# 正解ラベルの変換
answers = stock_data.answer.values
# 説明変数の変換
stock_data.drop(columns=['answer'], inplace=True)
explanatory_variable = stock_data.values

In [5]:
# データの正規化        
ms = MinMaxScaler()
ms.fit(explanatory_variable)
explanatory_variable = ms.transform(explanatory_variable)

In [6]:
# データの分割（データの80%を訓練用に、20％をテスト用に分割する）
X_train, X_test, y_train, y_test = train_test_split(explanatory_variable, answers, test_size=0.2, random_state=1,shuffle = False)

In [7]:
# グリッドサーチするパラメータを設定
parameters = {'C':[0.01,0.1,1,10,100],'loss':['hinge', 'squared_hinge']}
# グリッドサーチを実行
lsvc =  LinearSVC(random_state=1)
grid_search = GridSearchCV(lsvc, param_grid=parameters, cv=5)
grid_search = grid_search.fit(X_train , y_train)

# グリッドサーチ結果(最適パラメータ)を取得
GS_C, GS_loss = grid_search.best_params_.values()
print ("最適パラメータ：{}".format(grid_search.best_params_))

最適パラメータ：{'C': 0.1, 'loss': 'hinge'}


In [8]:
# 最適パラメーターを指定して学習
clf = LinearSVC(loss=GS_loss, C=GS_C, random_state=1)
clf.fit(X_train , y_train)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=1, tol=0.0001, verbose=0)

In [9]:
# 学習後のモデルによるテスト
# トレーニングデータを用いた予測
y_train_pred = clf.predict(X_train)
# テストデータを用いた予測
y_val_pred = clf.predict(X_test)

In [10]:
# 正解率の計算
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_val_pred)
# 正解率を表示
print("トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("テストデータに対する正解率：" + str(test_score * 100) + "%")

トレーニングデータに対する正解率：56.37270177625429%
テストデータに対する正解率：50.93399750933998%


In [11]:
#  交差検証
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable, answers, cv = 5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

Cross-Validation scores: [0.52428394 0.49315068 0.51681196 0.50997506 0.50811486]
Average score: 0.5104672988232137


In [12]:
# 学習結果の保存
joblib.dump(clf, 'lsvc.pkl')

['lsvc.pkl']